In [4]:
!pip install -q -U google-genai


In [1]:
import socket
import requests.packages.urllib3.util.connection as urllib3_cn

# Garder une référence à la fonction originale
_original_getaddrinfo = socket.getaddrinfo

def patched_getaddrinfo(*args, **kwargs):
    """Force la résolution d'adresses en IPv4."""
    responses = _original_getaddrinfo(*args, **kwargs)
    # Filtrer pour ne garder que les adresses IPv4 (socket.AF_INET)
    return [res for res in responses if res[0] == socket.AF_INET]

# Appliquer le patch globalement pour la résolution DNS via socket
socket.getaddrinfo = patched_getaddrinfo

# Appliquer aussi le patch pour urllib3 (utilisé par certaines bibliothèques http)
# Cela peut aider si d'autres appels HTTP indirects sont faits, bien que
# le problème principal ici soit gRPC.
urllib3_cn.allowed_gai_family = lambda: socket.AF_INET

print("Patch appliqué pour forcer l'utilisation d'IPv4.")

Patch appliqué pour forcer l'utilisation d'IPv4.


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
import os
from dotenv import load_dotenv

# Charger les variables depuis le fichier .env dans l'environnement du processus Python actuel
load_dotenv()

# Maintenant, essayez de lire la variable d'environnement
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("La variable d'environnement GOOGLE_API_KEY n'a pas pu être chargée (vérifiez le fichier .env).")
else:
    print("Clé API chargée avec succès via le fichier .env.")
    print(f"Clé API : {GOOGLE_API_KEY[:4]}...")
    # Configurez genai ici, maintenant que la clé est chargée
    try:
        import google.generativeai as genai
        genai.configure(api_key=GOOGLE_API_KEY)
        print("Configuration de GenAI réussie.")
    except ImportError:
        print("Erreur: Le module google.generativeai n'est pas installé ou importé.")
    except Exception as e:
        print(f"Erreur lors de la configuration de genai : {e}")

Clé API chargée avec succès via le fichier .env.
Clé API : AIza...
Configuration de GenAI réussie.


In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-e

In [6]:
model = genai.GenerativeModel('models/gemini-2.5-pro-exp-03-25')

In [7]:

response = model.generate_content("What is the meaning of life?")

In [8]:
to_markdown(response.text)

> That's arguably *the* fundamental question humans have pondered across cultures and millennia! There's **no single, universally agreed-upon answer**, and the meaning of life is often considered subjective and deeply personal.
> 
> However, we can explore the different ways people approach this question:
> 
> **1. Philosophical Perspectives:**
> 
> *   **Nihilism:** Argues that life has no inherent, objective meaning, purpose, or intrinsic value.
> *   **Existentialism:** Agrees there's no pre-ordained meaning, but emphasizes that individuals are free and responsible for *creating* their own meaning through their choices and actions. (Think Sartre, Camus).
> *   **Hedonism:** Suggests the meaning of life is to maximize pleasure and minimize pain.
> *   **Stoicism:** Focuses on living virtuously, in accordance with reason and nature, accepting what we cannot control. Meaning comes from inner resilience and ethical action.
> *   **Humanism:** Emphasizes human potential, reason, ethics, and flourishing in this life, often focusing on contributing to the greater good of humanity without recourse to the supernatural.
> 
> **2. Religious & Spiritual Perspectives:**
> 
> *   **Theistic Religions (e.g., Christianity, Islam, Judaism):** Often propose that meaning comes from serving a higher power (God), fulfilling divine commandments, achieving salvation or enlightenment, and living in relationship with the creator and fellow beings.
> *   **Eastern Religions (e.g., Buddhism, Hinduism):** May focus on achieving enlightenment, breaking the cycle of rebirth (samsara), understanding one's true nature (Atman/Buddha-nature), living in harmony, and reducing suffering through compassion and detachment.
> *   **Spirituality (Broader sense):** Finding meaning through connection to something larger than oneself – nature, the universe, a collective consciousness, or an inner sense of purpose.
> 
> **3. Scientific & Biological Perspectives:**
> 
> *   From a purely biological standpoint, the "purpose" of life is survival and reproduction – passing on genetic material.
> *   Some scientists and thinkers find meaning in the universe's capacity to develop complexity and consciousness – seeing humans as a way the universe experiences itself.
> *   However, science primarily describes *how* life works, not necessarily its subjective *meaning* or ultimate *why*.
> 
> **4. Psychological & Personal Perspectives:**
> 
> *   **Finding Purpose:** Many find meaning through having clear goals, pursuing passions, or dedicating themselves to a cause larger than themselves.
> *   **Connection & Relationships:** Love, family, friendship, and community provide profound meaning for many people.
> *   **Contribution & Service:** Making a positive impact on others or the world.
> *   **Growth & Learning:** Continuously developing oneself, gaining knowledge and wisdom.
> *   **Experience & Appreciation:** Finding meaning in the simple act of experiencing life, beauty, joy, and even overcoming challenges.
> *   **Legacy:** Creating something lasting or influencing future generations.
> *   **Logotherapy (Viktor Frankl):** Argues that the primary human drive is not pleasure, but the discovery and pursuit of what we personally find meaningful, which can be found through work, love, and courage in suffering.
> 
> **In Conclusion:**
> 
> Instead of a single answer, the "meaning of life" might be:
> 
> *   **Something you discover:** Through introspection, experience, or faith.
> *   **Something you create:** Through your choices, actions, and commitments.
> *   **A combination of both.**
> *   **Found in the journey itself:** The process of living, learning, connecting, and seeking might be where the meaning lies, rather than a final destination or definition.
> 
> Ultimately, the question "What is the meaning of *my* life?" is one that each individual must grapple with and answer for themselves. What gives *your* life meaning and purpose?

In [1]:
# Cellule 1: Installation
!pip install -q diffusers transformers accelerate ftfy scipy imageio gradio python-dotenv matplotlib

In [2]:
# Cellule 2: Imports
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, UNet2DConditionModel
from transformers import CLIPTextModel, CLIPTokenizer
from torch import optim, nn
from tqdm.auto import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os
from dotenv import load_dotenv
import imageio # Pour les GIFs

# Configuration initiale (GPU, etc.)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
import socket
import requests.packages.urllib3.util.connection as urllib3_cn
import functools # Bonne pratique pour le wrapping

print("Tentative d'application du patch IPv4...")

try:
    # Vérifier si le patch est déjà appliqué pour éviter les doubles patchs
    # (On vérifie si la fonction actuelle a le nom qu'on lui a donné)
    if getattr(socket.getaddrinfo, '__qualname__', '') == 'patched_getaddrinfo':
         print("Le patch IPv4 semble déjà appliqué.")
    else:
        # Garder une référence à la fonction originale
        _original_getaddrinfo = socket.getaddrinfo
        print(f"Fonction getaddrinfo originale : {_original_getaddrinfo}")

        @functools.wraps(_original_getaddrinfo) # Préserve les métadonnées de la fonction originale
        def patched_getaddrinfo(*args, **kwargs):
            """Force la résolution d'adresses en IPv4 en filtrant les résultats."""
            # print(f"Appel de patched_getaddrinfo avec args: {args}, kwargs: {kwargs}") # Pour débogage
            try:
                # ----- APPELER L'ORIGINAL SAUVEGARDÉ ICI -----
                responses = _original_getaddrinfo(*args, **kwargs)
                # ---------------------------------------------

                # Filtrer pour ne garder que les adresses IPv4 (socket.AF_INET)
                ipv4_responses = [res for res in responses if res[0] == socket.AF_INET]

                # print(f"Réponses originales: {len(responses)}, Réponses IPv4: {len(ipv4_responses)}") # Pour débogage

                # Que faire si aucune adresse IPv4 n'est trouvée ?
                # Retourner une liste vide pourrait casser certaines choses.
                # Pour l'instant, on retourne seulement les IPv4 si elles existent.
                # if not ipv4_responses:
                #     print("Avertissement: Aucune adresse IPv4 trouvée après filtrage.")
                #     # Option: retourner les réponses originales pour ne pas tout casser ?
                #     # return responses

                return ipv4_responses
            except Exception as e:
                print(f"Erreur à l'intérieur de patched_getaddrinfo : {e}")
                # En cas d'erreur dans le patch, peut-être revenir à l'original ?
                # return _original_getaddrinfo(*args, **kwargs)
                raise # Ou simplement relancer l'erreur

        # Appliquer le patch globalement
        socket.getaddrinfo = patched_getaddrinfo
        print(f"Fonction getaddrinfo patchée installée : {socket.getaddrinfo}")

        # Appliquer aussi le patch pour urllib3 (utilisé par certaines bibliothèques http)
        # Vérifier si déjà patché pour éviter les erreurs
        if not hasattr(urllib3_cn, '_original_allowed_gai_family'):
             urllib3_cn._original_allowed_gai_family = getattr(urllib3_cn, 'allowed_gai_family', None) # Sauvegarde si existe
             urllib3_cn.allowed_gai_family = lambda: socket.AF_INET
             print("Patch urllib3 allowed_gai_family appliqué.")
        else:
             print("urllib3 allowed_gai_family semble déjà patché.")

        print("Patch IPv4 appliqué avec succès.")

except Exception as e:
    print(f"Échec de l'application du patch IPv4 : {e}")
    # Optionnel : essayer de restaurer l'original si le patch a échoué à mi-chemin
    if '_original_getaddrinfo' in locals() and hasattr(socket, 'getaddrinfo') and getattr(socket.getaddrinfo, '__qualname__', '') == 'patched_getaddrinfo':
         socket.getaddrinfo = _original_getaddrinfo
         print("Tentative de restauration de getaddrinfo original.")
    if '_original_allowed_gai_family' in getattr(urllib3_cn, '__dict__', {}):
         urllib3_cn.allowed_gai_family = urllib3_cn._original_allowed_gai_family
         print("Tentative de restauration de allowed_gai_family original.")

2025-04-03 01:06:31.906677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743635191.927049   50997 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743635191.932950   50997 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-03 01:06:31.952947: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using device: cuda
Tentative d'application du patch IPv4...
Fonction getaddrinfo originale : <function getaddrinfo at 0x7f1b28978cc0>
Fonction getaddrinfo patchée installée : <function getaddrinfo at 0x7f199d9afd80>
Patch urllib3 allowed_gai_family appliqué.
Patch IPv4 appliqué avec succès.


In [4]:
# Cellule 3: Chargement du modèle Stable Diffusion
model_id = "stable-diffusion-v1-5/stable-diffusion-v1-5" # Ou un autre modèle SD

# Charger les composants
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder").to(device)
vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae").to(device) # Ajout de l'import
unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet").to(device)
scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler") # Utiliser DDIM comme dans Imagic

print("Modèle Stable Diffusion chargé.")

OSError: Can't load tokenizer for 'stable-diffusion-v1-5/stable-diffusion-v1-5'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'stable-diffusion-v1-5/stable-diffusion-v1-5' is the correct path to a directory containing all relevant files for a CLIPTokenizer tokenizer.

In [ ]:
# Cellule 4: Préparation de l'entrée
input_image_path = "path/to/your/image.jpg" # Mettez le chemin de votre image
input_image_pil = Image.open(input_image_path).convert("RGB").resize((512, 512))

target_prompt = "A photo of a cat wearing a party hat" # Votre texte cible

# Fonction pour prétraiter l'image pour le VAE
def preprocess(image):
    w, h = image.size
    w, h = map(lambda x: x - x % 32, (w, h)) # ensure resolution is multiple of 32
    image = image.resize((w, h), resample=Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.0 * image - 1.0

input_image_tensor = preprocess(input_image_pil).to(device)
# Obtenir la latente initiale de l'image d'entrée (utilisée pour la perte de reconstruction)
with torch.no_grad():
    input_latents = vae.encode(input_image_tensor).latent_dist.sample() * vae.config.scaling_factor # scaling_factor ~0.18215

plt.imshow(input_image_pil)
plt.title("Input Image")
plt.show()

In [ ]:
# Cellule 5: Étape A - Optimisation de l'Embedding

# Obtenir e_tgt (non entraînable)
text_input_tgt = tokenizer(target_prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
with torch.no_grad():
    e_tgt = text_encoder(text_input_tgt.input_ids.to(device))[0]

# Initialiser e_opt comme clone de e_tgt, mais entraînable
e_opt = e_tgt.clone().detach().requires_grad_(True)

# Configuration de l'optimisation
optimizer = optim.Adam([e_opt], lr=1e-3) # Ajuster lr
num_optimization_steps = 100 # Comme dans Imagic (~100), ajuster si besoin
unet.eval() # Garder UNet gelé pendant cette étape
text_encoder.eval() # Garder text_encoder gelé
vae.eval() # Garder VAE gelé

print("Début de l'optimisation de l'embedding...")
pbar = tqdm(range(num_optimization_steps))
for step in pbar:
    optimizer.zero_grad()

    # Échantillonner un timestep aléatoire
    t = torch.randint(0, scheduler.config.num_train_timesteps, (1,), device=device).long()

    # Bruiter l'image latente d'entrée
    noise = torch.randn_like(input_latents)
    latents_noisy = scheduler.add_noise(input_latents, noise, t)

    # Prédire le bruit avec e_opt
    noise_pred = unet(latents_noisy, t, encoder_hidden_states=e_opt).sample

    # Calculer la perte de reconstruction
    loss = nn.functional.mse_loss(noise_pred, noise, reduction="mean")

    loss.backward()
    optimizer.step()

    pbar.set_description(f"Loss: {loss.item():.4f}")

print("Optimisation de l'embedding terminée.")
e_opt_optimized = e_opt.clone().detach() # Sauvegarder l'embedding optimisé

In [ ]:
# Cellule 6: Étape B - Fine-tuning UNet (Simplifié - Envisager LoRA en pratique)

# Rendre UNet entraînable, geler le reste
unet.train()
text_encoder.eval()
vae.eval()
# e_opt_optimized n'a pas besoin de grad ici

optimizer_unet = optim.Adam(unet.parameters(), lr=5e-6) # Très petit lr pour fine-tuning
num_finetune_steps = 200 # Ajuster (~1500 dans Imagic, mais long/coûteux)

print("Début du fine-tuning de UNet...")
pbar = tqdm(range(num_finetune_steps))
for step in pbar:
    optimizer_unet.zero_grad()

    t = torch.randint(0, scheduler.config.num_train_timesteps, (1,), device=device).long()
    noise = torch.randn_like(input_latents)
    latents_noisy = scheduler.add_noise(input_latents, noise, t)

    # Prédire le bruit avec le UNet actuel et e_opt_optimized
    noise_pred = unet(latents_noisy, t, encoder_hidden_states=e_opt_optimized).sample

    loss = nn.functional.mse_loss(noise_pred, noise, reduction="mean")

    loss.backward()
    optimizer_unet.step()

    pbar.set_description(f"UNet Loss: {loss.item():.4f}")

print("Fine-tuning UNet terminé.")
# Sauvegarder les poids fine-tunés si nécessaire (ou les poids LoRA)
# unet.save_pretrained("./unet_finetuned")
unet.eval() # Remettre en mode évaluation

In [ ]:
# Cellule 7: Étape C - Interpolation et Génération

eta = 0.7 # Facteur d'interpolation (entre 0 et 1) - à ajuster
guidance_scale = 7.5 # Force de la guidance (CFG)
num_inference_steps = 50 # Nombre d'étapes de sampling

# Calculer l'embedding interpolé
e_bar = eta * e_tgt + (1 - eta) * e_opt_optimized

# Préparer pour CFG (Classifier-Free Guidance)
uncond_input = tokenizer([""] * 1, padding="max_length", max_length=tokenizer.model_max_length, return_tensors="pt")
with torch.no_grad():
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(device))[0]

text_embeddings = torch.cat([uncond_embeddings, e_bar])

# Initialiser la latente de départ (bruit pur)
latents = torch.randn((1, unet.config.in_channels, 512 // 8, 512 // 8), device=device) # Taille pour SD 1.5
latents = latents * scheduler.init_noise_sigma

# Boucle de sampling DDIM
scheduler.set_timesteps(num_inference_steps)

print("Début de la génération de l'image éditée...")
for t in tqdm(scheduler.timesteps):
    # Expansion des latentes pour CFG
    latent_model_input = torch.cat([latents] * 2)
    latent_model_input = scheduler.scale_model_input(latent_model_input, t)

    # Prédire le bruit
    with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

    # CFG
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

    # Calculer l'étape précédente de la latente
    latents = scheduler.step(noise_pred, t, latents).prev_sample

print("Génération terminée.")

# Décoder l'image finale
latents = 1 / vae.config.scaling_factor * latents
with torch.no_grad():
    image = vae.decode(latents).sample
image = (image / 2 + 0.5).clamp(0, 1)
image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
edited_image_pil = Image.fromarray((image * 255).astype(np.uint8))

# Afficher l'image éditée
plt.imshow(edited_image_pil)
plt.title(f"Edited Image (eta={eta})")
plt.show()

In [ ]:
# Cellule 8 (Concept): Contrôle par Attention Croisée

# --- Cette partie est complexe et nécessite une compréhension profonde ---
# --- de Prompt-to-Prompt et de l'architecture de CrossAttention de diffusers ---

# 1. Créer des classes AttentionProcessor personnalisées
class SaveAttentionProcessor:
    def __init__(self):
        self.attention_maps = {} # Dictionnaire pour stocker les cartes

    def __call__(self, attn, hidden_states, encoder_hidden_states=None, attention_mask=None):
        # ... logique pour calculer l'attention normalement ...
        # SAUVEGARDER la carte d'attention calculée (attn.attention_probs)
        # dans self.attention_maps avec une clé unique (ex: nom de couche, timestep)
        # ... retourner la sortie de l'attention ...
        return # output

class InjectAttentionProcessor:
    def __init__(self, saved_maps_source):
        self.saved_maps = saved_maps_source # Référence aux cartes sauvegardées

    def __call__(self, attn, hidden_states, encoder_hidden_states=None, attention_mask=None):
        # ... début du calcul de l'attention avec le prompt cible ...
        # RÉCUPÉRER la carte d'attention source correspondante depuis self.saved_maps
        # MODIFIER la carte d'attention cible en utilisant la carte source
        # (ex: remplacement partiel basé sur les tokens, etc.)
        # ... finir le calcul de l'attention avec la carte modifiée ...
        # ... retourner la sortie de l'attention ...
        return # output

# 2. Modifier la boucle de sampling (Étape C)

# -- Au début de la boucle de sampling --
# att_saver = SaveAttentionProcessor()
# unet.set_attn_processor(att_saver) # Attacher le saver

# latents_source = latents.clone() # Bruit initial pour la passe source

# -- Boucle sur les timesteps t --

    # --- Passe Source (pour sauvegarder l'attention) ---
    # latent_model_input_source = scheduler.scale_model_input(latents_source, t)
    # with torch.no_grad():
    #     # Utiliser e_opt_optimized pour la passe source
    #     _ = unet(latent_model_input_source, t, encoder_hidden_states=e_opt_optimized).sample
    # # Ici, att_saver.attention_maps devrait être rempli pour ce timestep

    # --- Passe Cible (avec injection d'attention) ---
    # att_injector = InjectAttentionProcessor(att_saver.attention_maps)
    # unet.set_attn_processor(att_injector) # Attacher l'injector

    # latent_model_input_target = torch.cat([latents] * 2) # Pour CFG
    # latent_model_input_target = scheduler.scale_model_input(latent_model_input_target, t)

    # with torch.no_grad():
    #     # Utiliser e_tgt (ou e_bar) pour la passe cible
    #     noise_pred = unet(latent_model_input_target, t, encoder_hidden_states=text_embeddings).sample # text_embeddings utilise e_tgt/e_bar

    # --- Fin de la passe cible ---
    # unet.set_attn_processor(None) # Détacher les processors (ou remettre les originaux)

    # ... reste de la logique CFG et scheduler.step(noise_pred, t, latents) ...

# --- Fin de la boucle ---

# 3. Décoder l'image finale comme avant
# ...